# Исследование надежности заемщиков.

**Цель исследования** — ответить на четыре вопроса и сделать общий вывод:

1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?


### Шаг 1. Обзор данных

In [ ]:
import pandas as pd
import numpy as np
df =  pd.read_csv('/datasets/data.csv')
# Анализирую таблицу при помощи метода info()
df.info()
# Нахожу долю пропусков от общего значения
days_gap_ratio = df['days_employed'].isna().sum() / len(df['days_employed'])
income_gap_ratio = df['total_income'].isna().sum() / len(df['total_income'])




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
df.isna().mean()

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

**Выводы:**
* Пропущенные значения представляют из себя `NaN`, `NaN` замещает отсутствующее в ячейке число и принадлежит к типу `float`.
* В столбцах `days_employed` и `total_income` присутствуют пропуски в одинаковом количестве – 2174. Доля оных составляет ~10%, что составляет больше 1% => просто удалить соответствующие строки нельзя, поскольку это повлияет на результат исследования.
* Пропуски могли появится по техническим, либо человеческим причинам.

In [ ]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Шаг 2.1 Заполнение пропусков

Для количественных переменных пропуски заполняются характерными знаениями. В этом случае подходит заполнение средним, потому что оно не сильно отличается от медианного значения.

In [ ]:
# средний заработок
income_median = df['total_income'].median()
# замена NaN на значение income_mean
df['total_income'] = df['total_income'].fillna(value=income_median)
# проверка заполнения total_income income_mean
df.isna().sum()
# медианное значения срока работы
days_median = df['days_employed'].median()
# замена NaN на значение days_median
df['days_employed'] = df['days_employed'].fillna(value=days_median)
# проверка на успешное заполнение NaN
df.isna().sum()


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [ ]:
# Поиск аномалий
for col in df:
    print('---------------')
    print(col)
    print(sorted(df[col].unique()))
    print('---------------')
# убираем все артефакты при помощи функции abs
df['days_employed'] = df['days_employed'].apply(abs)
# медианное значения срока работы
days_median = df['days_employed'].median()
# замена NaN на значение days_median
df['days_employed'] = df['days_employed'].fillna(value=days_median)
# проверка на успешное заполнение NaN
df['days_employed'].isna().sum()
# Заменим 0 значения в dob_years на медианное
age_median = df['dob_years'].median()
df.loc[df['dob_years'] == 0, 'dob_years'] = age_median
# Отредактируем артефакты в столбце children
df.loc[df['children'] == 20, 'children'] = 2
df['children'] = df['children'].apply(abs)
# Замена строк, где присутствует XNA в таблице gender на unkown
df.loc[df['gender'] == 'XNA', 'gender'] = 'unknown'


---------------
children
[-1, 0, 1, 2, 3, 4, 5, 20]
---------------
---------------
days_employed
[-18388.949900568383, -17615.563265627912, -16593.472817263817, -16264.699500887124, -16119.68773669392, -15835.725774811905, -15785.678893355003, -15773.0613349239, -15618.063786344595, -15410.040779095478, -15267.541182734109, -15193.032201443106, -15079.216068566157, -14920.049805042863, -14747.887934053397, -14677.9022934147, -14652.093785926449, -14583.15250401094, -14492.543986842333, -14447.661078047651, -14420.005180584703, -14381.545843648422, -14240.932399924128, -14103.863413698578, -14051.20262056069, -13976.806877265004, -13967.98538897768, -13923.375086079388, -13913.41587629005, -13894.357289777596, -13840.953867998049, -13809.045625637127, -13724.223883918732, -13710.428182355554, -13702.636708175783, -13664.197881775632, -13623.026318165186, -13563.100420050008, -13489.372942477548, -13468.882696747742, -13459.725039589568, -13424.333742690078, -13372.43469299878, -13359.0

In [ ]:
# Проверка
for col in df:
    print('---------------')
    print(col)
    print(sorted(df[col].unique()))
    print('---------------')

---------------
children
[0, 1, 2, 3, 4, 5]
---------------
---------------
days_employed
[24.14163324048118, 24.240694791435672, 30.19533715555962, 33.52066457593984, 34.701044500282805, 37.72660206855514, 39.95417043834918, 46.95279289413868, 47.1098397454324, 50.12829786804968, 50.48028820183397, 51.49688495716076, 54.00833410011489, 55.47564829593528, 55.838005521394265, 58.27648071486708, 60.6373275553343, 61.5184565937786, 61.596442746578425, 62.10848678222075, 62.79627506554634, 65.16319032061023, 67.03662472052932, 69.36814052410932, 70.09170994128691, 70.7815276880608, 72.62578520446468, 72.95782045845813, 73.98989065754823, 74.99524988670981, 75.14877255155419, 75.8974029174435, 75.98143820404016, 78.08254710399822, 79.09935516760508, 79.83206377973707, 79.88034967664467, 80.78330821160098, 81.57168061350069, 82.5269559600968, 82.6728710038733, 83.01076227576483, 83.67070938979354, 84.87542571409523, 85.27966797152988, 85.28272332936633, 85.29944871036041, 86.00617816580055, 

**Выводы:**


Возможные причины появлений аномалий:
* В столбце `days_employed` - человеческая: могли случайно домножить на -1
* В столбце `children` - человеческий фактор при написании: 1 - случайно нажать на `-` и 2 - нажать на `0`
* В столбце `family_status` - человеческий фактор: написание через клавишу `Shift`
* В столбце `dob_years` - человечески-технический фактор: на местах, где стоят `0` могли быть пропущенные значения, которые на будущее заменили на `0`и которые могли возниктуть из-за техничеких факторов.
* В столбце `gender` присутствует 1 аномалия - гендер `XNA`. Возможно и техническая и человеческая причина. Человек мог заполнить пропуск этим значением

### Шаг 2.3. Изменение типов данных.

In [ ]:
# замена типа
try:
    df['total_income'] = df['total_income'].astype(int)
except Exception as ex:
    print('Ошибка, нужно проверить тип данных')
try:
    df['dob_years'] = df['dob_years'].astype(int)
except Exception as ex:
    print('Ошибка, нужно проверить тип данных')
# проверка
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Шаг 2.4. Удаление дубликатов.

In [ ]:
# Ищем неявные дубликаты через метод lower()
df['education'] = df['education'].str.lower()
# Сумма дубликатов
df.duplicated().sum()
# Удаляем дубликаты и обновляем индексацию
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [ ]:
# Проверка
df.duplicated().sum()

0

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [ ]:
education_dict = df['education_id'].unique()

In [ ]:
education_dict

array([0, 1, 2, 3, 4])

In [ ]:
# "Словарь" образования
education_dict = df[['education','education_id']].drop_duplicates().reset_index(drop=True)
# "Словарь" семейного положения
family_dict = df[['family_status','family_status_id']].drop_duplicates().reset_index(drop=True)
display(family_dict.head(10))
display(education_dict.head(10))


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [ ]:
# Удаляю ненужные столбцы
del df['family_status']
del df['education']

### Шаг 2.6. Категоризация дохода.

In [ ]:
# Функция категоризация дохода
def category(row):
    if row['total_income'] <= 30000:
        return 'E'
    elif 30001 <= row['total_income'] <= 50000:
        return 'D'
    elif 50001 <= row['total_income'] <= 200000:
        return 'C'
    elif 200001 <= row['total_income'] <= 1000000:
        return 'B'
    return 'A'
# приминение функции
df['category_of_income'] = df.apply(category,axis=1)
# проверка
df['category_of_income'].value_counts()


C    16015
B     5041
D      350
A       25
E       22
Name: category_of_income, dtype: int64

**Выводы:**
Больше всего кредитов берут люди с зарплатой от 5о.ооо до 1.ооо.ооо.

### Шаг 2.7. Категоризация целей кредита.

In [ ]:
# Анализирую, какие строки подойдут для всех упоминай отдельных видов целей
df['purpose'].unique()
def purpose_foo(row):
    if 'жи' in row['purpose']:
        return 'операции с недвижимостью'
    elif 'авто'in row['purpose']:
        return 'операции с автомобилем'
    elif 'браз' in row['purpose']:
        return 'получение образования'
    return 'проведение свадьбы'
# применяю функцию к нужному dataframe
df['purpose_category'] = df.apply(purpose_foo, axis=1)
df

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,category_of_income,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...
21448,1,4529.316663,43,1,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью
21449,0,343937.404131,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем
21450,1,2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью
21451,3,3112.481705,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем


In [ ]:
df['purpose_category'].value_counts()

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2323
Name: purpose_category, dtype: int64

**Выводы**
Чаще всего берут кредиты на недвижимость. Почти в три раза реже кредит на автомобиль и образование. Реже всего на свадьбу

### Ответы на вопросы.

#### Вопрос 1:
##### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [ ]:
df_pivot_children = df.pivot_table(index='children',values='debt',aggfunc=['count','sum','mean'],fill_value=0)
df_pivot_children.columns = ['Всего','Просрочки','Отношение']
df_pivot_children.sort_values(by='Отношение',ascending=False)


,Всего,Просрочки,Отношение
children,,,
4,41,4,0.097561
2,2128,202,0.094925
1,4855,445,0.091658
3,330,27,0.081818
0,14090,1063,0.075444
5,9,0,0.000000


### С 1, 2 и 4 детьми -  максимальная вероятность к просрочки
Группы с 1, 2 и 4 детьми просрачивают кредит примерно одинаково часто. Однако лидерами являются семья с 4 детьми. Процент не уплаты ~9.1%, ~9.5%, ~9.8% соответственно.

### С 3-мя детьми
Редко просрачивают кредит люди с 3-мя детьми. процент пропуска ~8.2%.
### Без детей:
Из таблицы видно, что люди без детей просрачивают выплату по кредиту реже всех, чем остальные - ~7.5%.

### 5 детей - минимальная вероятность к просрочки
В семьях с 5 детьми просрочек не наблюдается.

### Вывод 1:
Прямой зависимости между количеством детей и возвратом кредита в срок нет. Однако есть тенденция, что люди, имеющие от 1 до 4 детей чаще всех не возвращают кредиты в срок - лидерами являются люди с 4-мя детьми. Исключением по количеству детей в этой базе данных являются люди с 5-ью детьми - оные люди платят все в срок, и не имеют ни одной просрочки

#### Вопрос 1:
##### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
df_pivot_family=df.pivot_table(index='family_status_id',values='debt',aggfunc=['count','sum','mean'])
df_pivot_family.columns = ['Всего','Просрочки','Отношение']
df_pivot_family = df_pivot_family.merge(family_dict,on='family_status_id',how='inner')
df_pivot_family.sort_values(by='Отношение',ascending=False)


,family_status_id,Всего,Просрочки,Отношение,family_status
4,4,2810,274,0.097509,Не женат / не замужем
1,1,4150,388,0.093494,гражданский брак
0,0,12339,931,0.075452,женат / замужем
3,3,1195,85,0.071130,в разводе
2,2,959,63,0.065693,вдовец / вдова



### Гражданский брак  и не женат / не замужем - максимальная вероятность к просрочки
Люди в гражданском браке и не женатые не платят по кредиту в срок чаще остальных - на ~2%, проценты по не уплате составляют ~9.3% и ~9.8% соответственно.
### Люди в разводе	и женат / замужем
Делят 2 место по просрочке за кредит ~7.1% и ~7.5% соответственно.
### Вдоцев / вдова - минимальная вероятность к просрочки
Реже всего просрачивают выплату по кредиту - ~6.6%.

### Вывод 2:
Зависимость между семейным положением и возвратом кредита в срок есть:
Люди, которыее не женаты / не замужем являются лидерами по просрочке выплаты по кредиту. Дальше по убыванию частоты идут люди в гражданском браке, женаты/замумеж, в разводе и вдовы / вдовцы.

In [ ]:
# Создам главное отношение для категорий дохода
income_dict = pd.DataFrame({'category_of_income': ['A','B','C','D','E'],
                           'income':['>1.000.001','200.001-1.000.000','50.001-200.000','30.001-50.000', '<30.000']})
income_dict

,category_of_income,income
0,A,>1.000.001
1,B,200.001-1.000.000
2,C,50.001-200.000
3,D,30.001-50.000
4,E,<30.000


In [ ]:
df_pivot_category_of_income=df.pivot_table(index='category_of_income',values='debt',aggfunc=['count','sum','mean'])
df_pivot_category_of_income.columns = ['Всего','Просрочки','Отношение']
df_pivot_category_of_income = df_pivot_category_of_income.merge(income_dict,on='category_of_income',how='inner')
df_pivot_category_of_income.sort_values(by='Отношение',ascending=False)

,category_of_income,Всего,Просрочки,Отношение,income
4,E,22,2,0.090909,<30.000
2,C,16015,1360,0.084920,50.001-200.000
0,A,25,2,0.080000,>1.000.001
1,B,5041,356,0.070621,200.001-1.000.000
3,D,350,21,0.060000,30.001-50.000


### E: <30.000 - максимальная вероятность к просрочки
Малозарабатывающие люди чаще всех просрачивают выплату по кредиту - ~9%
### C: 50.001 - 200.000
Люди среднего заработка занимают 2-ое место по просрочке кредита - ~8.5%
### A: >1.000.000
Самые состоятельные заемщики занимают третье место по просрочке - 8%.
### B: 200.001 - 1.000.000
Люди, имеющие достаточно хорошую зарплату, реже просрачивают выплату по кредиту ~7%.
### D: 30.001 - 50.000 -  минимальная вероятность к просрочки
Реже всех просрачивают кредит люди с зарплатой от 30 тысяч рублей до 50 тысяч рублей - 6%

### Вывод 3:
Таким образом, люди с наименьшей зарплатой являются лидерами по не оплате кредита в срок. Однаком прямой закономерности нет. Люди со средней зарплатой от 30 тысяч рублей до 50 тысяч рублей реще всех категорий просрачивают выплаты.

In [ ]:
df_pivot_purpose_category=df.pivot_table(index='purpose_category',values='debt',aggfunc=['count','sum','mean'])
df_pivot_purpose_category.columns = ['Всего','Просрочки','Отношение']
df_pivot_purpose_category.sort_values(by='Отношение',ascending=False)

,Всего,Просрочки,Отношение
purpose_category,,,
операции с автомобилем,4306,403,0.093590
получение образования,4013,370,0.092200
проведение свадьбы,2323,186,0.080069
операции с недвижимостью,10811,782,0.072334


### Вывод 4:

Чаще всего люди берут кредит на операции с недвижимостью
На 2 месте получение образование и проведение свадьбы
Однако по количеству невозврата первое место делят категории операции с автомобилем и получение образования.

## Общий вывод:

### Влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок ?

In [ ]:
df_pivot_final = df.pivot_table(index=['children'], columns=['family_status_id'],values='debt',aggfunc=['count','sum','mean'], fill_value=0)
display(df_pivot_final)
family_dict

count                        sum                        mean  \
family_status_id     0     1    2    3     4    0    1   2   3    4         0   
children                                                                        
0                 7468  2729  847  784  2262  516  229  53  55  210  0.069095   
1                 3004  1000   81  316   454  247  118   7  21   52  0.082224   
2                 1582   355   24   83    84  148   33   3   8   10  0.093552   
3                  249    56    6   11     8   17    8   0   1    1  0.068273   
4                   29     8    1    1     2    3    0   0   0    1  0.103448   
5                    7     2    0    0     0    0    0   0   0    0  0.000000   

                                                          
family_status_id         1         2         3         4  
children                                                  
0                 0.083914  0.062574  0.070153  0.092838  
1                 0.118000  0.086420  0.066456  0.114537  
2                 0.092958  0.125000  0.096386  0.119048  
3                 0.142857  0.000000  0.090909  0.125000  
4                 0.000000  0.000000  0.000000  0.500000  
5                 0.000000  0.000000  0.000000  0.000000

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Да, чаще просрачивают выплаты люди в гражданском браке и не женат / не замужем

In [ ]:
# Зависимость количества детей, категории заработка
df_income_children = df.pivot_table(index='children', columns='category_of_income',values='debt',aggfunc=['count','sum','mean'],fill_value=0)
df_income_children

count                       sum                       mean  \
category_of_income     A     B      C    D   E   A    B    C   D  E         A   
children                                                                        
0                     12  3245  10573  243  17   1  220  828  12  2  0.083333   
1                      9  1175   3600   67   4   1   88  352   4  0  0.111111   
2                      3   522   1568   34   1   0   43  155   4  0  0.000000   
3                      1    88    236    5   0   0    4   22   1  0  0.000000   
4                      0     7     34    0   0   0    1    3   0  0  0.000000   
5                      0     4      4    1   0   0    0    0   0  0  0.000000   

                                                            
category_of_income         B         C         D         E  
children                                                    
0                   0.067797  0.078313  0.049383  0.117647  
1                   0.074894  0.097778  0.059701  0.000000  
2                   0.082375  0.098852  0.117647  0.000000  
3                   0.045455  0.093220  0.200000  0.000000  
4                   0.142857  0.088235  0.000000  0.000000  
5                   0.000000  0.000000  0.000000  0.000000

###### Портрет клиента с наибольшей вероятности просрочки кредита:
* Семейный статус: не женат / не замужем, гражданский брак
* Количество детей: 1,2,4
* Доход: <30.000 или 50.001 - 200.000
* Цель кредита: операции с автомобилем или получение образования

###### Портрет клиента с наименьшей вероятности просрочки кредита:
* Семейный статус: Вдовец / вдова
* Количество детей: 0
* Доход: 30.001 - 50.000
* Цель кредита: операции с недвижимостью